# 2.1.1 Dynamical Systems #

## Equilibria of dynamical systems ##

In order to discuss equilibria of dynamical systems, the canonical model of gradient frequency neural networks (Large, Almonte, & Velasco, 2010) is introduced. This model is simple and is poised at a Hopf bifurcation (thus has rich dynamical properties).  
Under external periodic forcing, the canonical model of gradient frequency neural networks is (Kim & Large, 2015): 
  
$$\dot{z} = z\bigg(\alpha + i\omega + \beta_1 |z|^2 + \frac{\epsilon \beta_2 |z|^4}{1-\epsilon |z|^2}\bigg) + Fe^{i \omega_0 t}$$
  
where $Fe^{i \omega_0 t}$ is a periodic input with force $F \in \mathbf{R}$. $\omega$ and $\omega_0$ are the radian frequency of oscillation for the system and the external input, respectively. z is a complex state variable containing instantaneous amplitude and phase information ($z = re^{i\phi}$), and the parameters that determine the dynamical properties of the oscillator are $\alpha$, $\beta_1$, $\beta_2$, $\epsilon$ $\in \mathbf{R}$.  

The system in polar coordinates becomes: 

$$    
    \left\{
        \begin{array}{ll}
            \dot{r} = \alpha r + \beta_1 r^3 + \frac{\epsilon \beta_2 r^5}{1-\epsilon r^2} + F \cos(\psi)\\
            \dot{\psi} = \Omega - \frac{F}{r}\sin(\psi)\\
        \end{array}
    \right.
$$

Where $\Omega = \omega - \omega_0$ and $\psi = \phi - \omega_o t$  ($\phi$ is the angle of the initial state of the system).  
  
To understand the behavior of the system in its equilibrium points, the Jacobian matrix for the steady state $r^{*}$ $(\dot{r} = 0)$ must be calculated:
  
$$ L = D_xF = \bigg( \frac{\delta F_i}{\delta X_j} \bigg)_{i,j=1,...,m} = 
    \left\{
        \begin{array}{ll}
            \frac{\delta \dot{r}}{\delta r} & \frac{\delta \dot{r}}{\delta \psi}\\
            \frac{\delta \dot{\psi}}{\delta r} & \frac{\delta \dot{\psi}}{\delta \psi}\\
        \end{array}
    \right.
    = \\
    \left\{
        \begin{array}{ll}
            \alpha + 3\beta_1 r^{*2} + \frac{\epsilon \beta_2 r^{*4}(5-3\epsilon r^{*2})}{(1-\epsilon r^{*2})^2} & -F\sin(\psi^{*})\\
            \frac{F}{r^{*2}}\sin(\psi^{*}) & -\frac{F}{r^{*}}\cos(\psi^{*})\\
        \end{array}
    \right.
$$
  
The following program computes the Jacobian matrix symbolically:

In [1]:
import sympy as sp

# define the system:
r_star, psi_star, a, b_1, b_2, epsilon, Omega, F = sp.symbols('r_star, psi_star, a, b_1, b_2, epsilon, Omega, F')
R = a*r_star + b_1*r_star**3 + (epsilon*b_2*r_star**5)*(1-epsilon*r_star**2)**(-1) + F*sp.cos(psi_star)
P = Omega - ((F)*(r_star)**(-1))*sp.sin(psi_star)

# set equations to zero
R_star = sp.Eq(R, 0)
C_star = sp.Eq(P, 0)

# compute the Jacobian-matrix  
J = sp.Matrix([ R, P ]).jacobian(sp.Matrix([ r_star, psi_star ]))
print('The Jacobian Matrix is:')
print(J[0,0], J[0,1])
print(J[1,0], J[1,1])

The Jacobian Matrix is:
(a + 3*b_1*r_star**2 + 2*b_2*epsilon**2*r_star**6/(-epsilon*r_star**2 + 1)**2 + 5*b_2*epsilon*r_star**4/(-epsilon*r_star**2 + 1), -F*sin(psi_star))
(F*sin(psi_star)/r_star**2, -F*cos(psi_star)/r_star)


### Example 1 ###

However, to carry out a thorough analysis of the system's equilibra, we must consider cases where the parameters $\alpha$, $\beta_1$, $\beta_2$, $\epsilon$, $F$ and $\Omega$ are defined. Take as an example the case of the system at a limit-cycle, where $\alpha = 1$, $\beta_1 = -100$, $\beta_2 = 0$, $\epsilon = 0$, $F=0.02$ and $\Omega = 0$

In [2]:
# parameters for the system at a limit-cycle
a = 1
b_1 = -100
b_2 = 0
epsilon = 0
omega = 1.0*2*sp.pi
omega_0 = 1.0*2*sp.pi
Omega = omega - omega_0
F = 0.02

# define the system:
r_star, psi_star = sp.symbols('r_star, psi_star')
R = a*r_star + b_1*r_star**3 + (epsilon*b_2*r_star**5)*(1-epsilon*r_star**2)**(-1) + F*sp.cos(psi_star)
P = Omega - ((F)*(r_star)**(-1))*sp.sin(psi_star)

# set equations to zero
R_star = sp.Eq(R, 0)
C_star = sp.Eq(P, 0)

# compute the Jacobian-matrix 
J = sp.Matrix([ R, P ]).jacobian(sp.Matrix([ r_star, psi_star ]))
print('The Jacobian Matrix is:')
print(J[0,0], J[0,1])
print(J[1,0], J[1,1])

The Jacobian Matrix is:
(-300*r_star**2 + 1, -0.02*sin(psi_star))
(0.02*sin(psi_star)/r_star**2, -0.02*cos(psi_star)/r_star)


In [3]:
# compute fixed points (this might take a while)
eq_pts = sp.solve((R_star, C_star), r_star, psi_star, negative=False)

# compute the eigenvalues
for point in eq_pts:
    ev_mat = J.subs([(r_star, point[0]),(psi_star, point[1])])
    print('The equilibrium point r_star = %s, psi_star = %s has eigenvalues %s and %s. The determinant is %s' 
          %(point[0], point[1], ev_mat.eigenvals().keys()[0], ev_mat.eigenvals().keys()[1], ev_mat.det()))

The equilibrium point r_star = -0.108803391469129 + 0.e-22*I, psi_star = 3.14159265358979 has eigenvalues -2976067829382584480131707147975676648101067/2176067829382580000000000000000000000000000 - 3*sqrt(3635505317810124507006023460843511079190280526953533851857864806922447684912942817415200949563286610990881761)/4831909385789605409371436180000000000000000000000000000 and -2976067829382584480131707147975676648101067/2176067829382580000000000000000000000000000 + 3*sqrt(3635505317810124507006023460843511079190280526953533851857864806922447684912942817415200949563286610990881761)/4831909385789605409371436180000000000000000000000000000. The determinant is 2.76480172042079e-22*I/(-0.108803391469129 + 0.e-22*I) + 5.99903913064743e-36/(0.0118381779951845 - 4.60800286736798e-23*I) - 0.0510290679711071/(-0.108803391469129 + 0.e-22*I)
The equilibrium point r_star = -0.0878885066249973 + 0.e-22*I, psi_star = 0.0 has eigenvalues -1317316879031658287806767502187/1000000000000000000000000000000 and 

The eigenvalues shown above all have non-zero real parts, which makes the Jacobian be hyperbolic. For some of the equilibrium points, the Jacobian is hyperbolic stable (i.e. the real part is negative for all eigen values). An example of this is the last equilibrium point $r^{*} \approx 0.109$ and $\psi^{*} = 0.0$.  
Moreover, none of them has a determinant equal to zero, which makes them nonsingular.  
Finally, since the system seems to have equilibria that are hyperbolic stable, the system can be considered to be locally hyperbolic stable at these equilibria. However, as a cautionary note one should keep in mind that in a given system some equilibria may lead to Jacobian eigenvalues that are not hyperbolic stable.

### Example 2 ###
The case of the Hopf bifurcatio can also be studied with the same system, but with parameters $\alpha = 0$, $\beta_1 = 0$, $\beta_2 = 0$, $\epsilon = 0$, $F=0$ and $\Omega = 0$

In [4]:
# parameters for the system at a Hopf Bifurcation
a = 0
b_1 = 0
b_2 = 0
epsilon = 0
omega = 1.0*2*sp.pi
omega_0 = 0.98*2*sp.pi
Omega = omega - omega_0
F = 0.02

# define the system:
r_star, psi_star = sp.symbols('r_star, psi_star')
R = a*r_star + b_1*r_star**3 + (epsilon*b_2*r_star**5)*(1-epsilon*r_star**2)**(-1) + F*sp.cos(psi_star)
P = Omega - ((F)*(r_star)**(-1))*sp.sin(psi_star)

# set equations to zero
R_star = sp.Eq(R, 0)
C_star = sp.Eq(P, 0)

# compute the Jacobian-matrix 
J = sp.Matrix([ R, P ]).jacobian(sp.Matrix([ r_star, psi_star ]))
print('The Jacobian Matrix is:')
print(J[0,0], J[0,1])
print(J[1,0], J[1,1])

The Jacobian Matrix is:
(0, -0.02*sin(psi_star))
(0.02*sin(psi_star)/r_star**2, -0.02*cos(psi_star)/r_star)


In [5]:
# compute fixed points (this might take a while)
eq_pts = sp.solve((R_star, C_star), r_star, psi_star, negative=False)

print eq_pts

# compute the eigenvalues
for point in eq_pts:
    ev_mat = J.subs([(r_star, point[0]),(psi_star, point[1])])
    print('The equilibrium point r_star = %s, psi_star = %s has eigenvalues %s and %s. The determinant is %s' 
          %(point[0], point[1], ev_mat.eigenvals().keys()[0], ev_mat.eigenvals().keys()[1], ev_mat.det()))

[(-0.159154943091895, 4.71238898038469), (0.159154943091895, 1.57079632679490)]
The equilibrium point r_star = -0.159154943091895, psi_star = 4.71238898038469 has eigenvalues -46168096649323/4000000000000000000000000000000 - sqrt(252661872667887679999999999999997868506851778770380363641671)*I/4000000000000000000000000000000 and -46168096649323/4000000000000000000000000000000 + sqrt(252661872667887679999999999999997868506851778770380363641671)*I/4000000000000000000000000000000. The determinant is 0.0157913670417430
The equilibrium point r_star = 0.159154943091895, psi_star = 1.57079632679490 has eigenvalues -192367069372179/50000000000000000000000000000000 - sqrt(39478417604357449999999999999999962994910621159272114780791959)*I/50000000000000000000000000000000 and -192367069372179/50000000000000000000000000000000 + sqrt(39478417604357449999999999999999962994910621159272114780791959)*I/50000000000000000000000000000000. The determinant is 0.0157913670417430


If we disregard the error due to numerical approximation, we see that the eigenvalues for the Jacobian above are completely imaginary, which makes them non-hyperbolic. They are still non-singular, as the determinant of the Jacobian is not equal to zero.